In [3]:
from psycopg_pool import ConnectionPool
import os

In [4]:
pool = ConnectionPool(
    os.getenv("SUPABASE_CONECTION_URL")
)

In [7]:
with pool.connection() as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM financelive.users")
        rows = cur.fetchall()
        for row in rows:
            print(row)

(1, 'testuser0@test.com', 'testuser', 'lost', datetime.date(2003, 1, 29), 'test', datetime.datetime(2025, 3, 30, 1, 38, 36, 467736))


In [8]:
def use_connection(sql: str, params=None):
    if params is None:
        print("no params")
    with pool.connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params)
            rows = cur.fetchall()
            for row in rows:
                print(row)

In [10]:
## PROBAR LAS CONSULTAS
sql = """SELECT * FROM financelive.users WHERE email = %s"""
params = ("testuser0@test.com", )
use_connection(sql, params)

(1, 'testuser0@test.com', 'testuser', 'lost', datetime.date(2003, 1, 29), 'test', datetime.datetime(2025, 3, 30, 1, 38, 36, 467736))
